In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

In [3]:
table_13 = pd.read_csv('./Data/epl_table_1314.csv')
table_14 = pd.read_csv('./Data/epl_table_1415.csv')
table_15 = pd.read_csv('./Data/epl_table_1516.csv')
table_16 = pd.read_csv('./Data/epl_table_1617.csv')
table_17 = pd.read_csv('./Data/epl_table_1718.csv')

In [8]:
# Season fixture
epl_detail_1314 = pd.read_csv('./Data/epl1314.csv')
epl_detail_1415 = pd.read_csv('./Data/epl1415.csv')
epl_detail_1516 = pd.read_csv('./Data/epl1516.csv')
epl_detail_1617 = pd.read_csv('./Data/epl1617.csv')
epl_detail_1718 = pd.read_csv('./Data/epl1718.csv')
epl_detail_1819 = pd.read_csv('./Data/epl1819.csv')

In [15]:
epl_detail_1314[['HomeTeam', 'AwayTeam']].head(2)

,HomeTeam,AwayTeam
0,Arsenal,Aston Villa
1,Liverpool,Stoke


In [14]:
table_13[['Team','H_Att', 'H_Def', 'A_Att', 'A_Def', 'HGF', 'HGA', 'AGA', 'AGF']].head(2)

,Team,H_Att,H_Def,A_Att,A_Def,HGF,HGA,AGA,AGF
0,Arsenal,1.204013,0.484581,1.409692,1.003344,36,11,30,32
1,Aston Villa,0.735786,1.277533,0.748899,1.070234,22,29,32,17


In [ ]:
def fill_df(epl_detail_data, pes_data):
    '''
    Pass RPIs to fixture dataframe.
    IndexError will be ignored.
    The model ignores relegation and promotion of teams.
    '''
    for team in epl_detail_data['HomeTeam']:
        try:
            ht_off = pes_data.loc[pes_data['Team'] == team, 'H_Att'].values[0] 
            ht_def = pes_data.loc[pes_data['Team'] == team, 'H_Def'].values[0]
            at_off = pes_data.loc[pes_data['Team'] == team, 'A_Att'].values[0]
            at_def = pes_data.loc[pes_data['Team'] == team, 'A_Def'].values[0]
            
            
        
        except IndexError:
            pass
        
    return epl_detail_data

In [10]:
def score_percentage(dataframe, hometeam, awayteam):
    home_avg = dataframe['HGF'].sum()/380
    away_avg = dataframe['HGA'].sum()/380
    
    home_score = float(dataframe[dataframe['Team'] == hometeam]['H_Att']) * float(dataframe[dataframe['Team'] == awayteam]['A_Def']) * home_avg
    away_score = float(dataframe[dataframe['Team'] == awayteam]['A_Att']) * float(dataframe[dataframe['Team'] == hometeam]['H_Def']) * away_avg
    print(home_score, away_score)
    score = []
    # maximum score for a team is 5
    for goals in range(0, 6):
        scores = {}
        scores['Home'] = (poisson.pmf(goals, home_score) * 100) # Hometeam score
        scores['Away'] = (poisson.pmf(goals, away_score) * 100) # away score

        if len(scores) == 2:
                    score.append(scores)
    score = pd.DataFrame(score, columns=(['Home', 'Away']))
    return score
    
score = score_percentage(table_13, 'Aston Villa', 'Arsenal')

1.1617672944904067 2.1516345930906566


In [11]:
score

,Home,Away
0,31.293265,11.629391
1,36.355492,25.022200
2,21.118311,26.919315
3,8.178187,19.306843
4,2.375288,10.385318
5,0.551906,4.469082


In [12]:
score.Home.sum(), score.Away.sum()

(99.8724483585423, 97.73214927723953)

In [13]:
# % of home team winning
# home score > away score
# home[1] * away[0]
# home[2] * away[0] + home[2] * away[1]
# home[3] * away[0] + home[3] * away[1] + home[3] * away[2]
# home[4] * away[0] + home[4] * away[1] + home[4] * away[2] + home[4] * away[3]
# home[5] * away[0] + home[5] * away[1] + home[5] * away[2] + home[5] * away[3]+  home[5] * away[4]
home_w = 0
away_w = 0
draw = 0
for home in range(1, len(score)):
    for away in range(0, home):
        home_w += (score['Home'][home] * score['Away'][away] / 10000)

for away in range(1, len(score)):
    for home in range(0, away):
        away_w += (score['Home'][home] * score['Away'][away] / 10000)

for home in range(0, len(score)):
    away = home
    draw += (score['Home'][home] * score['Away'][away] / 10000)


In [14]:
# should get all percentage and average 38 games
home_w, away_w, draw

(0.1965037658716798, 0.5768575313544966, 0.2027136059398688)

In [16]:
home_w + away_w + draw

0.9667049816826642